In [4]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
warnings.filterwarnings(action = 'ignore')
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet as wn
from sklearn.cluster import KMeans
from numpy import log
import os
import os.path

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nissani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/nissani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nissani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nissani/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
#Loding train data, sorting it, and cleaning it for submission purposes
description_train_path = os.path.dirname("/Users/nissani/Desktop/data/descriptions_train/")
nums = []
txt = ".txt"
description_txt_paths = []
for i in range(10000):
    nums.append(str(i))
    description_txt_paths.append((os.path.join(description_train_path, nums[i] + txt )))

descrip_txt_train = list()
for item in description_txt_paths:
    with open(item) as f:
        text = f.read()
        descrip_txt_train.append(text)

descrip_txt_train = map(lambda x: x.replace('\n', ' '), descrip_txt_train)

descrip_txt_train = list(descrip_txt_train)

image_train_path = os.path.dirname("/Users/nissani/Desktop/data/images_train/")
nums = []
jpg = ".jpg"
image_jpg_paths = []
image_train_index = []
for i in range(10000):
    nums.append(str(i))
    image_jpg_paths.append((os.path.join(image_train_path, nums[i] + jpg )))
    image_train_index.append(nums[i] + jpg)

features_train = pd.read_csv('/Users/nissani/Desktop/data/features_train/features_resnet1000_train.csv')
features_train = features_train.set_index(['images train'])
index = list(features_train.index)
index = [int(i.split('/')[-1][:-4]) for i in index]
features_train.index = index
features_train = features_train.sort_index()
features_train.head()
features_train_array = features_train.values

pooling_train = pd.read_csv('/Users/nissani/Desktop/data/features_train/features_resnet1000intermediate_train.csv')
pooling_train = pooling_train.set_index(['Images Train'])
index = list(pooling_train.index)
index = [int(i.split('/')[-1][:-4]) for i in index]
pooling_train.index = index
pooling_train = pooling_train.sort_index()
pooling_train.head()
pooling_train = pooling_train.values

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 991,Unnamed: 992,Unnamed: 993,Unnamed: 994,Unnamed: 995,Unnamed: 996,Unnamed: 997,Unnamed: 998,Unnamed: 999,Unnamed: 1000
0,-4.552913,-3.339373,0.739805,-1.064960,-0.385821,-1.374796,-3.172979,-1.216543,-4.122320,-0.473386,...,-2.021026,-4.752773,-4.072814,-3.873384,-3.553856,-2.005920,-2.187191,-4.325321,-1.361008,-0.184895
1,1.055800,1.345920,-4.621484,-3.391327,-2.474905,-0.765481,-3.783750,-3.479563,-0.968133,-2.054501,...,0.447561,-1.310622,-1.228498,-0.492747,0.690989,1.746380,3.944386,-0.335490,4.469384,-0.193547
2,-1.323054,-1.990459,-2.646253,-2.555408,0.194242,0.360438,-2.450288,-1.364834,-1.818168,-2.428344,...,-1.727636,-4.679168,-4.418219,-1.780263,-2.594645,-1.229027,0.516662,-3.346858,0.924743,-0.353018
3,1.642971,-0.591986,-2.008492,-1.683242,0.152383,-0.580315,-2.876101,-2.001029,-2.138638,-0.831547,...,2.253989,-1.823335,-0.135379,-2.881580,-1.125049,1.979692,0.571415,-2.186887,2.552499,2.634879
4,-0.690964,-3.641853,-1.879540,0.208626,0.236147,-3.154541,-3.594615,0.742241,-0.431021,-0.036614,...,-1.286898,-4.253161,-3.368766,-3.720367,-3.813512,-1.947907,-2.103985,-4.788396,-0.305454,1.142632


In [7]:
#Loding test data, sorting it, and cleaning it for submission purposes
description_test_path = os.path.dirname("/Users/nissani/Desktop/data/descriptions_test/")
nums = []
txt = ".txt"
description_txt_paths = []
for i in range(2000):
    nums.append(str(i))
    description_txt_paths.append((os.path.join(description_test_path, nums[i] + txt )))

descrip_txt_test = list()
for item in description_txt_paths:
    with open(item) as f:
        text = f.read()
        descrip_txt_test.append(text)

descrip_txt_test = map(lambda x: x.replace('\n', ' '), descrip_txt_test)

descrip_txt_test = list(descrip_txt_test)

nums = []
txt = ".txt"
descrip_test_index = []
for i in range(2000):
    nums.append(str(i))
    descrip_test_index.append(nums[i] + txt)
        
image_test_path = os.path.dirname("/Users/nissani/Desktop/data/images_test/")
nums = []
jpg = ".jpg"
image_jpg_paths = []
image_test_index = []
for i in range(2000):
    nums.append(str(i))
    image_jpg_paths.append((os.path.join(image_test_path, nums[i] + jpg )))
    image_test_index.append(nums[i] + jpg)
    
#Load feautres
features_test = pd.read_csv('/Users/nissani/Desktop/data/features_test/features_resnet1000_test.csv')
features_test = features_test.set_index(['images test'])
index = list(features_test.index)
index = [int(i.split('/')[-1][:-4]) for i in index]
features_test.index = index
features_test = features_test.sort_index()
features_test.head()
features_test_array = features_test.values

pooling_test = pd.read_csv('/Users/nissani/Desktop/data/features_test/features_resnet1000intermediate_test.csv')
pooling_test = pooling_test.set_index(['images test'])
index = list(pooling_test.index)
index = [int(i.split('/')[-1][:-4]) for i in index]
pooling_test.index = index
pooling_test = pooling_test.sort_index()
pooling_test.head()
pooling_test = pooling_test.values

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 991,Unnamed: 992,Unnamed: 993,Unnamed: 994,Unnamed: 995,Unnamed: 996,Unnamed: 997,Unnamed: 998,Unnamed: 999,Unnamed: 1000
0,-1.330675,0.274202,-1.102608,-1.115833,-0.636994,-0.565285,-4.464181,-1.352997,-2.201896,-3.087491,...,-3.200296,-1.880090,-0.901040,-0.698119,-0.341681,1.505841,2.476855,-1.551757,1.481907,0.211971
1,0.172611,-2.445323,-1.314998,-0.756428,-1.415201,0.201333,-3.961030,-0.046629,-0.447107,-0.085347,...,1.587954,-1.121916,-1.888536,-0.509820,-1.554114,2.041140,0.122266,-1.603565,1.794990,0.071381
2,1.432785,-2.007764,-0.813398,-1.540644,1.948337,1.599849,-1.288280,-2.507530,-3.011521,-3.066300,...,-3.025030,-3.969228,-3.580349,-1.667892,-1.994338,-0.892279,1.623393,-2.797277,1.546263,1.800777
3,-1.672061,-0.162291,-2.062806,-2.233102,-0.936919,0.312382,-2.414253,-1.297624,-2.854015,-2.378380,...,-2.542696,-2.370132,-3.813374,-2.103109,-2.178449,-1.669393,0.146124,-3.502493,-0.059849,-2.313099
4,-0.635521,3.801764,-1.771022,-1.244705,-1.574648,1.245530,-2.842735,-1.460044,-0.281990,-3.713153,...,1.929877,-2.210485,-3.229804,-1.886067,-3.056182,-0.802163,-0.456715,-4.077403,1.949248,1.637591


In [11]:
#Lowercase text
for i in range(len(descrip_txt_train)):
    descrip_txt_train[i] = descrip_txt_train[i].lower()

for i in range(len(descrip_txt_test)):
    descrip_txt_test[i] = descrip_txt_test[i].lower()

#Make a set of punctutation and numerals to remove from reviews
exclude = string.punctuation
exclude += '0'
exclude += '1'
exclude += '2'
exclude += '3'
exclude += '4'
exclude += '5'
exclude += '6'
exclude += '7'
exclude += '8'
exclude += '9'
exclude = set(exclude)

#Srtip punctuation from words.
for i in range(len(descrip_txt_train)):
    descrip_txt_train[i] = ''.join([ch if ch not in exclude else ' ' for ch in descrip_txt_train[i]])

for i in range(len(descrip_txt_test)):
    descrip_txt_test[i] = ''.join(ch if ch not in exclude else ' ' for ch in descrip_txt_test[i])

In [12]:
#tfidf from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import string

tfidf = TfidfVectorizer(analyzer='word', stop_words = 'english', sublinear_tf=True)
tfidf.fit(descrip_txt_train)
response = tfidf.transform(descrip_txt_train)
response_test = tfidf.transform(descrip_txt_test)
feature_names = tfidf.get_feature_names()

#print non zero words
doc = 0
feature_index = response[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [response[doc, x] for x in feature_index])
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
    print(w, s)

#final matrix
corpus_index = [n for n in descrip_txt_train]
final_matrix = response.todense()
descriptions = corpus_index
feature_words = feature_names

#final matrix_test
corpus_index_test = [n for n in descrip_txt_test]
final_matrix_test = response_test.todense()
descriptions_test = corpus_index_test

watching 0.16549585588554036
using 0.16650171777283726
tricks 0.19954436165526726
trick 0.17496302623762142
table 0.2783294087144789
stage 0.25888800000782686
skateboarder 0.31590005129611726
skateboard 0.26952329548920084
skate 0.17452889815062317
riding 0.19833271659078594
putting 0.21609745794813365
pulling 0.18847493416435052
picnic 0.5332649803096715
person 0.10302813784752404
man 0.08558491450291536
doing 0.16633198545269748
crowd 0.16582783429921344
boarder 0.2191779012692396


In [13]:
#Remove specific stop words that may have an effect on connoting sentences
stop_words = stopwords.words('english')
stop_words.remove('no')
stop_words.remove('against')
stop_words.remove('up')
stop_words.remove('down')
stop_words.remove('over')
stop_words.remove('under')
stop_words.remove('not')
stop_words.remove('nor')

stop_words = set(stop_words)
  
#Define a strip stops words function
def strip_stop_words(el):
    word_tokens = word_tokenize(el)
    
    no_stop_words_sentence = [w for w in word_tokens if w not in stop_words]
    
    return no_stop_words_sentence

#Strip out stop words from the reviews
for i in range(len(descrip_txt_train)):
    descrip_txt_train[i] = strip_stop_words(descrip_txt_train[i])

for i in range(len(descrip_txt_test)):
    descrip_txt_test[i] = strip_stop_words(descrip_txt_test[i])

#initialize a lemmatizer and a stemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

#Lemmatize and then stem words
for i in range(len(descrip_txt_train)):
    for j in range(len(descrip_txt_train[i])):
        word = descrip_txt_train[i][j]
        try: #If a word is not in the wordnet dictionary, set the part of speech to noun
            part_of_speech = wn.synsets(word)[0].pos()
        except:
            part_of_speech = 'n'
            
        lemmatized_word = lemmatizer.lemmatize(word, pos = part_of_speech)
        stem_word = stemmer.stem(lemmatized_word)
    
        descrip_txt_train[i][j] = stem_word

for i in range(len(descrip_txt_test)):
    for j in range(len(descrip_txt_test[i])):
        word = descrip_txt_test[i][j]
        try: #If a word is not in the wordnet dictionary, set the part of speech to noun
            part_of_speech = wn.synsets(word)[0].pos()
        except:
            part_of_speech = 'n'
            
        lemmatized_word = lemmatizer.lemmatize(word, pos = part_of_speech)
        stem_word = stemmer.stem(lemmatized_word)
    
        descrip_txt_test[i][j] = stem_word

In [14]:
unique_words = []

#Create a unique words dictionary
for sentence in descrip_txt_train:
    for word in sentence:
        if word not in unique_words:
            unique_words.append(word)
            
bag_of_words_train = np.zeros((len(descrip_txt_train), len(unique_words))) #columns are individual words in unique dictionary, rows are reviews
bag_of_words_test = np.zeros((len(descrip_txt_test), len(unique_words))) 

for sentence in descrip_txt_train: 
    for word in unique_words:
        if word in sentence:
            bag_of_words_train[descrip_txt_train.index(sentence)][unique_words.index(word)] += sentence.count(word)

for sentence in descrip_txt_test:
    for word in unique_words:
        if word in sentence:
            bag_of_words_test[descrip_txt_test.index(sentence)][unique_words.index(word)] += sentence.count(word)

#Removing unnecessary words
bow_test_clean = []
bow_train_clean = []
workable_train = bag_of_words_train.T
workable_test = bag_of_words_test.T

for i in range(len(workable_test)):
    if 1 in workable_test[i]:
        bow_test_clean.append(workable_test[i])
        bow_train_clean.append(workable_train[i])
        
bow_test_clean = np.array(bow_test_clean).T
bow_train_clean = np.array(bow_train_clean).T


In [18]:
#Log normalizing data. We found that this works a bit better, and allows for NB.
normalized_train = log(bow_train_clean + 1)
normalized_test = log(bow_test_clean + 1)
normalized_tfidf_train = log(final_matrix + 1)
normalized_tfidf_test = log(final_matrix_test + 1)

In [21]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [22]:
def variance_normalizer(x):
    var = []
    for el in x.T:
        if np.var(el) == 0:
            var.append(0)
        else:
            np.var(el)
    
    output = []
    for el in x:
        output.append(np.multiply(el, var))
    
    return np.array(output)

In [23]:
features_train_softmax = features_train_array
features_train_softmax_float = features_train_softmax.astype('float32')

features_softmax_scores_train = list()
for i in range(len(features_train_softmax_float)):
    features_softmax_scores_train.append(softmax(features_train_softmax_float[i]))

In [24]:
features_test_softmax = features_test_array
features_test_softmax_float = features_test_softmax.astype('float32')

features_softmax_scores_test = list()
for i in range(len(features_test_softmax_float)):
    features_softmax_scores_test.append(softmax(features_test_softmax_float[i]))

In [25]:
features_softmax_scores_train = np.array(features_softmax_scores_train)
features_softmax_scores_test = np.array(features_softmax_scores_test)

In [93]:
description_train = np.concatenate((normalized_train, normalized_tfidf_train), axis = 1)

In [94]:
resnet_train = np.concatenate((features_softmax_scores_train, pooling_train), axis = 1)

In [311]:
feature_matrix_train = np.concatenate((description_train, features_softmax_scores_train), axis = 1)

In [97]:
resnet_test = np.concatenate((features_softmax_scores_test, pooling_test), axis = 1)

In [98]:
description_test = np.concatenate((normalized_test, normalized_tfidf_test), axis = 1)

In [297]:
# K-means on descriptions
from sklearn.cluster import KMeans
k = 100
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(response)
class_labels = model.predict(response)

#print(class_prediction[:15])

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tfidf.get_feature_names()
top10_terms = list()
for i in range(k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        top10_terms.append(terms[ind])
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:
 bicyclists
 bus
 cyclists
 speeding
 driver
 stands
 pass
 street
 group
 city
Cluster 1:
 phone
 cell
 bench
 talking
 using
 young
 man
 photograph
 sitting
 boy
Cluster 2:
 building
 outside
 people
 gathering
 motorcycles
 eat
 sunny
 number
 brick
 crowd
Cluster 3:
 tennis
 court
 racquet
 swings
 man
 hit
 getting
 racket
 ready
 hand
Cluster 4:
 dining
 kitchen
 lon
 chairs
 adjacent
 room
 leading
 brown
 table
 counter
Cluster 5:
 surfboard
 beach
 ocean
 man
 holding
 wet
 suit
 surf
 standing
 board
Cluster 6:
 toilet
 stall
 tiling
 colored
 commode
 bathroom
 tiles
 enclosed
 black
 tile
Cluster 7:
 pizza
 table
 checkered
 cloth
 vegetable
 served
 place
 pan
 restaurant
 sitting
Cluster 8:
 tunnel
 train
 station
 track
 travels
 bridge
 pulling
 silver
 gray
 passenger
Cluster 9:
 cow
 mother
 baby
 holstein
 field
 grass
 eating
 pool
 graze
 cows
Cluster 10:
 couch
 wii
 remote
 video
 boy
 orange
 child
 shirt
 game
 playing
Cluster

 food
 plate
 meat
 fleet
 lettuce
 tomato
 vegetables
 various
 white
Cluster 132:
 restaurant
 boys
 sons
 booth
 sit
 father
 table
 drinks
 meal
 man
Cluster 133:
 tower
 clock
 pointed
 clocktower
 tall
 street
 sides
 busy
 middle
 stands
Cluster 134:
 kitchen
 sink
 cabinets
 counter
 cupboards
 dishwasher
 space
 lots
 window
 filled
Cluster 135:
 bears
 curled
 teddy
 stuffed
 cat
 surfed
 laying
 asleep
 pillow
 cats
Cluster 136:
 slice
 pizza
 plate
 pepperoni
 white
 table
 piece
 big
 close
 large
Cluster 137:
 toilet
 roll
 bathroom
 paper
 directly
 contains
 placed
 edge
 white
 corner
Cluster 138:
 books
 book
 store
 motorcycles
 shelf
 library
 kinds
 trains
 sit
 different
Cluster 139:
 baseball
 bat
 player
 home
 plate
 prepares
 standing
 hit
 getting
 holds
Cluster 140:
 cows
 cow
 grass
 grazing
 laying
 lounging
 field
 rests
 cattle
 distance
Cluster 141:
 couch
 dog
 living
 room
 cute
 sitting
 camera
 floor
 cat
 big
Cluster 142:
 mirror
 taking
 crazy
 su

Cluster 222:
 baseball
 swinging
 batter
 mates
 bat
 progress
 player
 ball
 game
 team
Cluster 223:
 london
 clock
 tower
 britain
 city
 massive
 ben
 photo
 towering
 old
Cluster 224:
 kitchen
 preparing
 cooking
 dinner
 man
 food
 cabinet
 crowded
 stove
 lights
Cluster 225:
 luggage
 rack
 locked
 holder
 kind
 appears
 suitcases
 bags
 pile
 metal
Cluster 226:
 bikes
 bicycles
 riding
 men
 scooters
 building
 roadway
 school
 adults
 large
Cluster 227:
 costume
 party
 dressed
 costumes
 people
 gathered
 group
 posing
 photo
 couple
Cluster 228:
 wedding
 cutting
 cake
 unto
 layer
 multi
 couple
 reaching
 knife
 making
Cluster 229:
 broccoli
 pizza
 cheese
 pan
 vegetables
 cheesy
 chicken
 cooked
 toppings
 topped
Cluster 230:
 purse
 cat
 handbag
 orange
 rests
 bag
 corner
 chair
 head
 floor
Cluster 231:
 cake
 fork
 plate
 piece
 slice
 bread
 pound
 bundt
 banana
 white
Cluster 232:
 posted
 rainier
 sign
 spencer
 signs
 street
 names
 roads
 points
 numbers
Cluster 

 sink
 stove
 counter
Cluster 404:
 statue
 elephant
 colorful
 park
 posed
 sculpture
 painting
 colors
 painted
 middle
Cluster 405:
 bears
 fighting
 hugging
 couple
 rock
 rocks
 leaning
 play
 enclosure
 river
Cluster 406:
 snow
 mountain
 skis
 skiis
 man
 skies
 pose
 skiing
 ski
 slope
Cluster 407:
 ceiling
 kitchen
 kichen
 angled
 loft
 skylight
 slanted
 sunlight
 shelves
 small
Cluster 408:
 giraffes
 tall
 habitat
 trees
 standing
 leaves
 animals
 giraffe
 stand
 background
Cluster 409:
 bottles
 wine
 smiling
 table
 man
 beer
 smiles
 dinner
 sitting
 outdoor
Cluster 410:
 cupcakes
 stacked
 shelf
 wrappers
 treats
 trays
 frosting
 collection
 slices
 dining
Cluster 411:
 toilet
 sink
 bathroom
 curtain
 room
 tub
 shower
 showing
 light
 white
Cluster 412:
 benches
 tables
 restaurant
 people
 rows
 drinks
 sitting
 talking
 long
 eating
Cluster 413:
 skateboard
 crouches
 skating
 road
 riding
 skateboarder
 moving
 man
 skate
 park
Cluster 414:
 teddy
 bear
 chair
 

 parked
 run
 runway
 setting
Cluster 495:
 frisbee
 playing
 grass
 people
 game
 men
 couple
 field
 standing
 fluted
Cluster 496:
 dressed
 wear
 formal
 photos
 couple
 dress
 drink
 standing
 gray
 posing
Cluster 497:
 whip
 chocolate
 pastry
 cream
 scones
 drizzled
 plate
 caption
 provide
 pastries
Cluster 498:
 billboard
 billboards
 slum
 poor
 rundown
 repair
 advertisement
 area
 country
 women
Cluster 499:
 delivery
 truck
 parked
 foreign
 chinese
 mini
 building
 box
 street
 store
Cluster 500:
 boat
 river
 pontoon
 going
 yacht
 sailing
 type
 water
 people
 white
Cluster 501:
 elephant
 man
 elephants
 seat
 gray
 head
 dirt
 background
 sitting
 sits
Cluster 502:
 fish
 screen
 cat
 television
 picture
 tv
 watching
 standing
 sitting
 fluted
Cluster 503:
 paddle
 surfboard
 beach
 kayak
 calm
 shoreline
 lay
 edge
 waves
 body
Cluster 504:
 beach
 surfboard
 man
 surf
 guy
 sitting
 board
 laying
 sits
 young
Cluster 505:
 salon
 dryer
 hair
 women
 smiling
 lady
 h

...

 surfers
 riding
 waves
 surfing
 rides
 surfboard
 ocean
Cluster 9627:
 double
 decker
 bus
 snowy
 snows
 headed
 street
 deck
 makes
 city
Cluster 9628:
 tower
 clock
 tall
 soar
 building
 clouds
 sides
 beautiful
 stone
 sunny
Cluster 9629:
 kite
 beach
 flying
 scenery
 laying
 man
 sand
 cloudy
 taking
 sky
Cluster 9630:
 tennis
 racquet
 step
 playing
 swings
 shorts
 giant
 reaching
 takes
 man
Cluster 9631:
 tennis
 player
 ball
 serves
 court
 shirt
 action
 shorts
 served
 just
Cluster 9632:
 rain
 umbrella
 boots
 waling
 raincoat
 flooded
 woman
 heavy
 walking
 walks
Cluster 9633:
 tower
 clock
 tall
 rocky
 building
 rocks
 seen
 pile
 displayed
 hill
Cluster 9634:
 cake
 piece
 eating
 girl
 princess
 table
 dress
 pink
 little
 yellow
Cluster 9635:
 clock
 tower
 weathervane
 vein
 weather
 steeple
 church
 building
 skies
 type
Cluster 9636:
 horses
 cart
 pulling
 wagon
 carriage
 hay
 group
 man
 carrying
 riding
Cluster 9637:
 skateboard
 trick
 ledge
 doing
 flip

 keyboard
 computer
 paws
 small
 playing
 walking
 young
 standing
 sitting
Cluster 9718:
 kite
 flying
 field
 backpack
 person
 lady
 tree
 sky
 grassy
 day
Cluster 9719:
 decker
 double
 bus
 curb
 headlights
 metropolitan
 dusk
 travelling
 red
 parked
Cluster 9720:
 tennis
 upset
 racket
 court
 celebrating
 player
 shorts
 striped
 man
 racquet
Cluster 9721:
 wave
 surfboard
 surfboarding
 crest
 riding
 man
 underneath
 surf
 board
 person
Cluster 9722:
 cows
 jersey
 field
 grass
 cattle
 pasture
 eat
 herd
 sit
 eating
Cluster 9723:
 sheep
 grazing
 herd
 pasture
 mountains
 grassy
 farm
 field
 lush
 animals
Cluster 9724:
 baseball
 bat
 catcher
 player
 swing
 swinging
 swung
 extended
 glove
 having
Cluster 9725:
 glove
 baseball
 boy
 hardwood
 shirt
 mitt
 ball
 hands
 kid
 small
Cluster 9726:
 seagull
 beach
 sand
 gull
 bird
 shells
 water
 sea
 type
 standing
Cluster 9727:
 skateboard
 ripped
 riding
 jeans
 ledge
 edge
 man
 trick
 skate
 person
Cluster 9728:
 umbrel

 pitch
 baseball
 mound
 player
 pitchers
 winding
 ball
 throw
 pose
 pitcher
Cluster 9809:
 tennis
 swinging
 ball
 racket
 motion
 serving
 mid
 man
 racquet
 crowd
Cluster 9810:
 desktop
 books
 desk
 laptop
 office
 computer
 organize
 keyboards
 monitor
 row
Cluster 9811:
 refrigerator
 kitchen
 stove
 microwave
 cabinets
 splash
 drawers
 appliances
 fridge
 tile
Cluster 9812:
 ramp
 skateboard
 grinds
 skate
 man
 rail
 skateboarding
 night
 going
 park
Cluster 9813:
 tennis
 ball
 hit
 player
 funny
 prepares
 trying
 swinging
 face
 racquet
Cluster 9814:
 tennis
 ball
 racket
 bit
 woman
 holding
 match
 serve
 prepares
 female
Cluster 9815:
 wii
 men
 actively
 video
 chucks
 nun
 living
 remotes
 controllers
 games
Cluster 9816:
 baseball
 swinging
 bat
 game
 plays
 batter
 watch
 kid
 player
 ball
Cluster 9817:
 narrow
 toilet
 sink
 bathroom
 mirror
 window
 vanity
 bath
 plants
 tiled
Cluster 9818:
 wedding
 cake
 cutting
 banquet
 cut
 newly
 married
 groom
 bride
 pre

 sitting
Cluster 9899:
 dryer
 blow
 hair
 girl
 smiling
 hairdryer
 drying
 young
 black
 holding
Cluster 9900:
 bench
 cat
 park
 patio
 wooden
 brown
 perched
 sitting
 stone
 black
Cluster 9901:
 pizza
 slice
 plate
 piece
 tablecloth
 table
 toppings
 eaten
 ready
 covered
Cluster 9902:
 soccer
 kicking
 children
 playing
 group
 ladies
 ball
 girls
 field
 sunny
Cluster 9903:
 mirror
 taking
 bathroom
 garment
 picture
 woman
 sweater
 takes
 red
 photo
Cluster 9904:
 trains
 station
 train
 ceiling
 pulled
 platform
 stopped
 underneath
 silver
 tracks
Cluster 9905:
 wii
 controller
 game
 living
 stepper
 video
 stair
 playing
 room
 controllers
Cluster 9906:
 tennis
 got
 ball
 racket
 court
 playing
 low
 end
 hitting
 man
Cluster 9907:
 control
 drinking
 remote
 cup
 bed
 sippy
 hand
 boy
 toddler
 drinks
Cluster 9908:
 frisbee
 mouth
 dog
 air
 jumping
 frisbees
 thrown
 jumps
 catching
 catch
Cluster 9909:
 tennis
 racket
 tenn
 court
 opponent
 returning
 player
 ball
 s

 players
 group
Cluster 9990:
 wave
 rides
 surfer
 surfboard
 tide
 sized
 rider
 wetsuit
 wet
 surfing
Cluster 9991:
 tennis
 racket
 court
 return
 holding
 man
 waiting
 racquet
 male
 fence
Cluster 9992:
 tennis
 court
 ball
 swinging
 racket
 hits
 hard
 hitting
 person
 male
Cluster 9993:
 frisbee
 throwing
 park
 tossing
 throws
 man
 grass
 plays
 green
 young
Cluster 9994:
 soccer
 girls
 playing
 competitive
 ladies
 kicking
 field
 group
 running
 women
Cluster 9995:
 cat
 laptop
 laying
 computer
 gray
 desk
 keyboard
 white
 open
 table
Cluster 9996:
 frisbee
 playing
 park
 game
 men
 group
 guys
 kids
 grassy
 field
Cluster 9997:
 tennis
 hits
 ball
 hitting
 racket
 player
 woman
 swinging
 racquet
 white
Cluster 9998:
 frisbee
 playing
 men
 field
 group
 young
 running
 people
 grassy
 park
Cluster 9999:
 wave
 surfboard
 ocean
 riding
 wetsuit
 wet
 surfer
 rides
 suit
 guy


In [312]:
model1 = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=0, max_features=113)
model1.fit(feature_matrix_train, class_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=113, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [313]:
model2 = MultinomialNB()
model2.fit(feature_matrix_train, class_labels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [314]:
clf1 = MultinomialNB()
clf2 = RandomForestClassifier(n_estimators=30, max_depth=5, random_state=0, max_features=113)
eclf = VotingClassifier(estimators = [('mvg',clf1),('RF',clf2)], voting = 'soft')
eclf.fit(feature_matrix_train, class_labels)

VotingClassifier(estimators=[('mvg', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)), ('RF', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=113, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [262]:
def test_sample(index, trained_model):
    test_sample = [description_test[index]]*len(features_softmax_scores_test)
    test_sample = np.array(test_sample)
    query_array = np.concatenate((test_sample, features_softmax_scores_test), axis = 1)
    
    prediction_probs = trained_model.predict_proba(query_array)
    
    max_probs = []
    for i in range(len(prediction_probs)):
        max_probs.append(max(prediction_probs[i]))
    
    sorted_indices = np.argsort(max_probs)
    sorted_indices = np.flip(sorted_indices)
    results = image_test_index[sorted_indices[:20]]
    image_test_results = ' '.join([word for word in results])
    
    return [descrip_test_index[index], image_test_results]

In [263]:
submit = []

for i in range(2000):
    submit.append(test_sample(i), eclf)

In [265]:
np.savetxt("LastTry.csv", submit, delimiter=',', header="Description_ID,Top_20_Image_IDs", fmt = "%s")